In [64]:
import pickle
import pandas as pd
import os
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import rdMolDraw2D
from PIL import Image
from IPython.display import SVG
import io

In [ ]:
# Define current working directory so we can navigate folders easily
cwd = os.getcwd()
print(cwd)
os.chdir('../')
pwd = os.getcwd()
print(pwd)

In [9]:
# Import all the results into a list of pandas dataframes
files = os.listdir(pwd+'/NN_results/')
pkl_files = [i for i in files if '.csv' not in i]
list_of_dfs = []
for file in pkl_files:
    d = pd.DataFrame(pd.read_pickle(os.path.join(pwd+'/NN_results/'+file))).transpose()
    d.columns = ['SMILES', 'Yield', 'Temp', 'Time']
    list_of_dfs.append(d)

In [29]:
# The order of the files imported
pkl_files

['Huisgen Cycloaddition_10_NN.pkl',
 'Heck Reaction_10_NN.pkl',
 'Suzuki-Miyaura Coupling_10_NN.pkl',
 'Grignard Reaction_10_NN.pkl',
 'Buchwald-Hartwig Coupling_10_NN.pkl']

In [30]:
# Example of the 10 NN results returned
list_of_dfs[0] # Huisgen Cycloadditions

,SMILES,Yield,Temp,Time
0,C#CC1=CC=CC=C1.FC(F)(F)C1=CC=C(CN=N#N)C=C1.[Na...,99.0,80.0,16
1,C#CC1=CC=CC=C1.N#N=NCC1=CC=C2C=CC=CC2=C1.[Na+]...,94.0,80.0,16
2,CC1=CC=C(CN=N#N)C=C1.C#C.[Na+].OC[C@H](O)C1OC(...,99.0,90.0,16
3,COC1=CC=C(CN=N#N)C=C1.C#C.[Na+].OC[C@H](O)C1OC...,95.0,90.0,16
4,FC(F)(F)C1=CC=C(CN=N#N)C=C1.C#C.[Na+].OC[C@H](...,94.0,90.0,16
5,FC1=CC=C(CN=N#N)C=C1.C#C.[Na+].OC[C@H](O)C1OC(...,99.0,90.0,16
6,N#CC1=CC=C(CN=N#N)C=C1.C#C.[Na+].OC[C@H](O)C1O...,99.0,90.0,16
7,N#N=NCC1=CC=C2C=CC=CC2=C1.C#C.[Na+].OC[C@H](O)...,99.0,90.0,16
8,N#N=NCC1=CC=CC=C1.C#C.[Na+].OC[C@H](O)C1OC(=O)...,92.0,90.0,16
9,O=N(=O)C1=CC=C(CN=N#N)C=C1.C#C.[Na+].OC[C@H](O...,99.0,90.0,16


In [15]:
# Read in the list of reactions which have a known reaction type
with_reaction_type = pd.read_csv(pwd+'/data/reactions_with_reaction_type.csv')

In [79]:
# Iterate through the list of results and find how many of the returned
# results have the same reaction class as the query reaction
not_present_df_list =[]
present_df_list = []
for index, df in enumerate(list_of_dfs):
    
    # Get the name of the reaction class
    reaction_class = pkl_files[index][:-10]
    
    # Make a directory to store the results
    os.makedirs(pwd+"/NN_results/"+reaction_class)
    
    # Select all the reactions which have the same reaction type as the query reaction
    known_reactions = with_reaction_type[with_reaction_type['Reaction Type']==reaction_class]
    print(pkl_files[index][:-10])
    print("Number of Reactions in class with label = "+str(len(known_reactions)))
    
    # See how many of the returned nearest neighbours are found within these reactions
    results = known_reactions[known_reactions['Reactants.Reagents>>Products'].isin(df.SMILES)]
    
    # Find the results which were not present in the list of classified reactions
    not_present = [i for i in df.SMILES.to_list() if i not in results['Reactants.Reagents>>Products'].to_list()]
    
    # Store the results in lists
    not_present_df = df[df.SMILES.isin(not_present)]
    not_present_df_list.append(not_present_df)
    present_df = df[df.SMILES.isin(results['Reactants.Reagents>>Products'])]
    present_df_list.append(present_df)
    
    # Save the results as csv files
    not_present_df.to_csv(pwd+"/NN_results/"+reaction_class+"/"+reaction_class+"_not_present.csv",index=False)
    present_df.to_csv(pwd+"/NN_results/"+reaction_class+"/"+reaction_class+"_present.csv",index=False)
    
    # Generate images of the reactions - NOT Present
    for index, smi in enumerate(not_present_df.SMILES):
        rxn = Chem.rdChemReactions.ReactionFromSmarts(smi, useSmiles=True)
        drawer = rdMolDraw2D.MolDraw2DCairo(800, 200)
        drawer.SetFontSize(0.75)
        drawer.DrawReaction(rxn)
        drawer.FinishDrawing()
        png = drawer.GetDrawingText()
        im = Image.open(io.BytesIO(png))
        im.save(pwd+"/NN_results/"+reaction_class+"/"+reaction_class+f"_not_present_{index}.png")
        
    # Generate images of the reactions - Present
    for index, smi in enumerate(present_df.SMILES):
        rxn = Chem.rdChemReactions.ReactionFromSmarts(smi, useSmiles=True)
        drawer = rdMolDraw2D.MolDraw2DCairo(800, 200)
        drawer.SetFontSize(0.75)
        drawer.DrawReaction(rxn)
        drawer.FinishDrawing()
        png = drawer.GetDrawingText()
        im = Image.open(io.BytesIO(png))
        im.save(pwd+"/NN_results/"+reaction_class+"/"+reaction_class+f"_present_{index}.png")
    
    
    print("Present in NN's = " +str(present_df.shape[0]))
    print("------------------------------")
    

Huisgen Cycloaddition
Number of Reactions in class with label = 30
Present in NN's = 10
------------------------------
Heck Reaction
Number of Reactions in class with label = 56
Present in NN's = 8
------------------------------
Suzuki-Miyaura Coupling
Number of Reactions in class with label = 69
Present in NN's = 10
------------------------------
Grignard Reaction
Number of Reactions in class with label = 9
Present in NN's = 1
------------------------------
Buchwald-Hartwig Coupling
Number of Reactions in class with label = 63
Present in NN's = 4
------------------------------


RDKit ERROR: [16:33:18] Explicit valence for atom # 7 B, 5, is greater than permitted
[16:33:18] Explicit valence for atom # 7 B, 5, is greater than permitted
RDKit ERROR: [16:33:18] Explicit valence for atom # 7 B, 4, is greater than permitted
[16:33:18] Explicit valence for atom # 7 B, 4, is greater than permitted
RDKit ERROR: [16:33:18] Explicit valence for atom # 7 B, 4, is greater than permitted
[16:33:18] Explicit valence for atom # 7 B, 4, is greater than permitted


# Generate images of the reactions we queried first to get an idea of how 
# representative they are of their respective raction class

In [22]:
# import the query reactions
query_reactions = pickle.load(open(pwd+'/data/query_reactions_drfp.pkl',"rb"))

In [74]:
# Generate images of the query reactions
query_rxns = []
for index, i in enumerate(query_reactions[1]):
    rxn = Chem.rdChemReactions.ReactionFromSmarts(query_reactions[1][index], useSmiles=True)
    drawer = rdMolDraw2D.MolDraw2DCairo(800, 200)
    drawer.SetFontSize(0.75)
    drawer.DrawReaction(rxn)
    drawer.FinishDrawing()
    png = drawer.GetDrawingText()
    im = Image.open(io.BytesIO(png))
    im.save(pwd+"/NN_results/"+query_reactions[2][index]+"_query.png")